# Proyecto | Base de Datos de Big Data
---

**MAESTRÍA EN INTELIGENCIA ARTIFICIAL APLICADA**

**Curso: TC4034.10 - Análisis de grandes volúmenes de datos**

Tecnológico de Monterrey

* Dr. Iván Olmos Pineda
* Mtra. Verónica Sandra Guzmán de Valle
* Mtro. Alberto Daniel Salinas Montemayor

**Proyecto**
Base de Datos de Big Data

---

**Equipo 37**

|  NOMBRE COMPLETO                        |     MATRÍCULA     | 
| :-------------------------------------: |:-----------------:|
| Alejandro Díaz Villagómez               |  A01276769        | 
| Alonso Pedrero Martínez                 |  A01769076        | 
| César Iván Pedrero Martínez             |  A01366501        | 
| Emiliano Saucedo Arriola                |  A01659258        | 

> Tenga en cuenta que para ejecutar este notebook, deberá instalar algunas dependencias. Se recomienda ejecutar este comando para instalarlo en un entorno virtual. Para ello, siga este comando:

```bash
python3 -m venv .venv && source .venv/bin/activate && pip install findspark pyspark setuptools kagglehub
```

In [1]:
# Activar PySpark en notebooks
import findspark
findspark.init()

# PySpark SQL
from pyspark.sql import SparkSession
from pyspark.sql import functions as F


### Definición de clases auxiliares

In [2]:
class FileManager():
    @staticmethod
    def open_csv_file(file_path: str):
        """
        This method opens a csv file with pyspark
        """
        csv_df = spark.read.csv(
            file_path,
            header=True,
            inferSchema=True,
            multiLine=True,
            escape="\"",
            quote="\""
        )

        return csv_df

## 0. Inicialización

In [3]:
findspark.init()
findspark.find()

'/Users/emisaar/anaconda3/envs/pyspark_env/lib/python3.10/site-packages/pyspark'

In [4]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

spark

25/05/01 22:00:00 WARN Utils: Your hostname, Emis-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.100.73 instead (on interface en0)
25/05/01 22:00:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/01 22:00:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Cargar Dataset

In [5]:
import kagglehub

# Download latest version
FILE_PATH = kagglehub.dataset_download("machharavikiran/amazon-reviews")

print("Path to dataset files:", FILE_PATH)

/Users/emisaar/anaconda3/envs/pyspark_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/emisaar/.cache/kagglehub/datasets/machharavikiran/amazon-reviews/versions/1


In [6]:
df_reviews = FileManager.open_csv_file(FILE_PATH)
df_reviews.limit(10).toPandas()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date,sentiment
0,US,22873041,R3ARRMDEGED8RD,B00KJWQIIC,335625766,Plemo 14-Inch Laptop Sleeve Case Waterproof Fa...,PC,5,0,0,N,Y,Pleasantly surprised,I was very surprised at the high quality of th...,2015-08-31,1
1,US,30088427,RQ28TSA020Y6J,B013ALA9LA,671157305,TP-Link OnHub AC1900 Wireless Wi-Fi Router,PC,5,24,31,N,N,OnHub is a pretty no nonsense type router that...,I am a Google employee and had to chance to us...,2015-08-31,1
2,US,20329786,RUXJRZCT6953M,B00PML2GQ8,982036237,AmazonBasics USB 3.0 A Male to A Male Cable - ...,PC,1,2,2,N,N,None of them worked. No functionality at all.,"Bought cables in 3ft, 6ft and 9ft. NONE of th...",2015-08-31,0
3,US,14215710,R7EO0UO6BPB71,B001NS0OZ4,576587596,Transcend P8 15-in-1 USB 2.0 Flash Memory Card...,PC,1,0,0,N,Y,just keep searching.,"nope, cheap and slow",2015-08-31,0
4,US,38264512,R39NJY2YJ1JFSV,B00AQMTND2,964759214,Aleratec SATA Data Cable 2.0 20in Serial ATA S...,PC,5,0,0,N,Y,Five Stars,Excellent! Great value and does the job.,2015-08-31,1
5,US,30548466,R31SR7REWNX7CF,B00KX4TORI,170101802,Kingston Digital MobileLite G4 USB 3.0 Multi-F...,PC,5,0,0,N,Y,"Good quality, works well and compact","Good quality,works well and compact size",2015-08-31,1
6,US,589298,RVBP8I1R0CTZ8,B00P17WEMY,206124740,White 9 Inch Unlocked Dual Sim Card Phone Tabl...,PC,3,1,2,N,Y,in fact this is third China good. Demn s***,This demn tablet is just a Real Chinese produc...,2015-08-31,0
7,US,49329488,R1QF6RS1PDLU18,B00TR05L9Y,778403103,"Lenovo TAB2 A10 - 10.1"" 2-in-1 Tablet (1.5Ghz,...",PC,4,1,1,N,Y,Good,I am not sure I don't know if it is the tablet...,2015-08-31,1
8,US,50728290,R23AICGEDAJQL1,B0098Y77OG,177098042,Acer,PC,1,0,0,N,Y,You get what you pay for,"After exactly 45 days, the screen went dark. P...",2015-08-31,0
9,US,37802374,R2EY3N4K9W19UP,B00IFYEYXC,602496520,AzureWave Broadcom BCM94352HMB 802.11/ac/867Mb...,PC,5,3,4,N,Y,Great for Windows 7 Laptop!,Replaced my Intel Centrino 2230 with the BCM94...,2015-08-31,1


## 1. Caracterización

### Análisis del mercado (`marketplace`)

Observaciones:
* El mercado se centra únicamente en US.

In [ ]:
df_reviews.groupBy("marketplace").agg(
    F.count("*").alias("count")
).orderBy(F.desc("count")).show()


+-----------+-------+
|marketplace|  count|
+-----------+-------+
|         US|6906564|
+-----------+-------+



### Análisis de categorías (`product_category`)

Observaciones:
* Las reseñas pertecen solamente a la categoría de PC.

In [ ]:

df_reviews.groupBy("product_category").agg(
    F.count("*").alias("count")
).orderBy(F.desc("count")).show()

+----------------+-------+
|product_category|  count|
+----------------+-------+
|              PC|6906564|
+----------------+-------+



### Análisis de clasificación de estrellas (`star_rating`)

Observaciones:
* Fuerte sesgo hacia reseñas positivas.
* Alrededor del 60% de las reseñas tienen 5 estrellas, lo que indica una tendencia general positiva.

In [ ]:
star_rating_counts = df_reviews.groupBy("star_rating").count().orderBy("star_rating")

star_rating_stats = df_reviews.agg(
    F.min("star_rating"), F.max("star_rating"),
    F.avg("star_rating"), F.stddev("star_rating")
)

star_rating_counts.show()
star_rating_stats.show()

+-----------+-------+
|star_rating|  count|
+-----------+-------+
|          1| 756857|
|          2| 362156|
|          3| 513656|
|          4|1168208|
|          5|4105687|
+-----------+-------+



+----------------+----------------+-----------------+-------------------+
|min(star_rating)|max(star_rating)| avg(star_rating)|stddev(star_rating)|
+----------------+----------------+-----------------+-------------------+
|               1|               5|4.086460937739808|  1.362854001234071|
+----------------+----------------+-----------------+-------------------+



### Análisis de votos útiles (`helpful_ratio`)

Observaciones:
* Los datos sugieren una alta variabilidad en la utilidad percibida de las reseñas.

In [ ]:
df_with_ratio = df_reviews.withColumn(
    "helpful_ratio", 
    F.when(F.col("total_votes") > 0, F.col("helpful_votes") / F.col("total_votes")).otherwise(0)
)

helpful_ratio_stats = df_with_ratio.agg(
    F.min("helpful_ratio"), F.max("helpful_ratio"),
    F.avg("helpful_ratio"), F.stddev("helpful_ratio")
)

helpful_ratio_stats.show()

+------------------+------------------+-------------------+---------------------+
|min(helpful_ratio)|max(helpful_ratio)| avg(helpful_ratio)|stddev(helpful_ratio)|
+------------------+------------------+-------------------+---------------------+
|               0.0|               1.0|0.23128070845636955|   0.3967627166047604|
+------------------+------------------+-------------------+---------------------+



### Análisis de votos totales (`total_votes`)

Observaciones:
* En promedio, las reseñas reciben pocos votos (1.96).
* Lo anterior sugiere que son pocas las reseñas que reciben gran cantidad de votos.

In [ ]:
total_votes_stats = df_with_ratio.agg(
    F.min("total_votes"), F.max("total_votes"),
    F.avg("total_votes"), F.stddev("total_votes")
)

total_votes_stats.show()

+----------------+----------------+------------------+-------------------+
|min(total_votes)|max(total_votes)|  avg(total_votes)|stddev(total_votes)|
+----------------+----------------+------------------+-------------------+
|               0|           48362|1.9620770907212328|  43.07079218821245|
+----------------+----------------+------------------+-------------------+



### Análisis de sentimiento (`sentiment`)

Observaciones:
* Se muestra una alta tendencia a sentimientos positivos. Posiblemente exista una alta correlación con el rating (`star_rating`).

In [ ]:
sentiment_counts = df_with_ratio.groupBy("sentiment").count().orderBy("sentiment")
sentiment_stats = df_with_ratio.agg(F.avg("sentiment").alias("proporcion_positiva"))

sentiment_counts.show()
sentiment_stats.show()

+---------+-------+
|sentiment|  count|
+---------+-------+
|        0|1632669|
|        1|5273895|
+---------+-------+



+-------------------+
|proporcion_positiva|
+-------------------+
| 0.7636061868101128|
+-------------------+



### Análisis de usuarios (Top 10 `customer_id`)

Observaciones:
* Algunos clientes han generado más de 350 reseñas, lo que podría señalar una actividad inusual o usuarios con cierta antigüedad en la plataforma.
* Esta exploración, permite comprender y detectar posibles sesgos individuales o comportamiento anómalo.

In [ ]:
customer_top10 = df_with_ratio.groupBy("customer_id").count().orderBy(F.desc("count")).limit(10)
customer_top10.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   17957446|  458|
|   44834233|  442|
|   52938899|  366|
|   45664110|  275|
|   49452274|  261|
|   50820654|  256|
|   12200139|  251|
|   45070473|  251|
|   32038204|  241|
|   49266466|  240|
+-----------+-----+



### Análisis de vine

Observaciones:
* Son pocas las reseñas que fueron incentivadas por el programa Vine de Amazon.

In [14]:
vine_counts = df_with_ratio.groupBy("vine").count()
total_reviews = df_with_ratio.count()

vine_proportions = vine_counts.withColumn(
    "proportion", F.round(F.col("count") / F.lit(total_reviews), 4)
)
vine_proportions.show()

+----+-------+----------+
|vine|  count|proportion|
+----+-------+----------+
|   Y|  36228|    0.0052|
|   N|6870336|    0.9948|
+----+-------+----------+



### Análisis de compras verificadas (`verified_purchase`)

Observaciones:
* Se puede apreciar una alta proporción de compras verificadas, lo cual sugiere la credibilidad de las reseñas provenientes de usuarios que realmente compraron el producto.

In [ ]:
verified_counts = df_with_ratio.groupBy("verified_purchase").count()
verified_proportions = verified_counts.withColumn(
    "proportion", F.round(F.col("count") / F.lit(total_reviews), 4)
)

verified_proportions.show()

+-----------------+-------+----------+
|verified_purchase|  count|proportion|
+-----------------+-------+----------+
|                Y|6047075|    0.8756|
|                N| 859489|    0.1244|
+-----------------+-------+----------+



### Análisis temporal por año (`review_year`)

Observaciones:
* Las reseñas han incrementado a través del tiempo, con un fuerte crecimiento a partir de 2013.

In [ ]:
df_with_year = df_with_ratio.withColumn("review_year", F.year("review_date"))
reviews_by_year = df_with_year.groupBy("review_year").count().orderBy("review_year")

reviews_by_year.show()

+-----------+-------+
|review_year|  count|
+-----------+-------+
|       1999|    384|
|       2000|   3596|
|       2001|   6588|
|       2002|  10125|
|       2003|  13619|
|       2004|  14124|
|       2005|  18171|
|       2006|  26277|
|       2007|  59870|
|       2008|  81409|
|       2009| 129840|
|       2010| 213170|
|       2011| 397182|
|       2012| 661742|
|       2013|1396819|
|       2014|1996666|
|       2015|1876982|
+-----------+-------+



Las variables de interés se seleccionaron con base en su relación con la percepción del cliente y la confiabilidad de la reseña. En particular, `star_rating` y `verified_purchase` mostraron una buena distribución de combinaciones y relevancia para el objetivo del proyecto, que es analizar el sentimiento de los clientes.

## 2. Particionamiento

Se genera una columna combinada (`partition_key`) para unificar `star_rating` y `verified_purchase`. Posteriormente se crea 10 particiones del dataset según las reglas de particionamiento definidas. Finalmente, se almacena cada partición en un diccionario y muestra su tamaño.

In [17]:
df_partitioned = df_reviews.withColumn(
    "partition_key",
    F.concat_ws("_", F.col("star_rating"), F.col("verified_purchase"))
)

partition_keys = [
    "1_N", "1_Y", "2_N", "2_Y", "3_N", "3_Y", "4_N", "4_Y", "5_N", "5_Y"
]

partitions = {}
for key in partition_keys:
    partitions[key] = df_partitioned.filter(F.col("partition_key") == key)
    print(f"Tamaño de la partición {key}: {partitions[key].count()}")

Tamaño de la partición 1_N: 153484


Tamaño de la partición 1_Y: 603373


Tamaño de la partición 2_N: 61607


Tamaño de la partición 2_Y: 300549


Tamaño de la partición 3_N: 70284


Tamaño de la partición 3_Y: 443372


Tamaño de la partición 4_N: 148437


Tamaño de la partición 4_Y: 1019771


Tamaño de la partición 5_N: 425677


Tamaño de la partición 5_Y: 3680010


In [18]:
partitions["5_Y"].show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+-------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|sentiment|partition_key|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------+-------------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|        1|          5_Y|
|         US|   38264512

In [19]:
# spark.stop()